## Mizzou vs Indiana State History

In [1]:
from urllib.request import urlopen
import pandas as pd

In [2]:
from bs4 import BeautifulSoup

In [3]:
years = []
for year in range(2009,2020):
    years.append(str(year))

In [4]:
base_schedule_url = 'https://gosycamores.com/sports/womens-soccer/schedule/'
schedule_urls = []
for year in years:
    isu_schedule_url = base_schedule_url+year
    schedule_urls.append(isu_schedule_url)

In [5]:
print(schedule_urls)

['https://gosycamores.com/sports/womens-soccer/schedule/2009', 'https://gosycamores.com/sports/womens-soccer/schedule/2010', 'https://gosycamores.com/sports/womens-soccer/schedule/2011', 'https://gosycamores.com/sports/womens-soccer/schedule/2012', 'https://gosycamores.com/sports/womens-soccer/schedule/2013', 'https://gosycamores.com/sports/womens-soccer/schedule/2014', 'https://gosycamores.com/sports/womens-soccer/schedule/2015', 'https://gosycamores.com/sports/womens-soccer/schedule/2016', 'https://gosycamores.com/sports/womens-soccer/schedule/2017', 'https://gosycamores.com/sports/womens-soccer/schedule/2018', 'https://gosycamores.com/sports/womens-soccer/schedule/2019']


In [6]:
isu_hist_schedule = pd.DataFrame(index=None)
opponent_list = []
date_list = []
result_list = []    
year_list = []
for schedule_url in schedule_urls:
    # empty lists

    
    # Set up parser
    page = urlopen(schedule_url)
    html = page.read().decode('utf-8')
    soup = BeautifulSoup(html,"html.parser")
    
    # pull info
    otags = soup.find_all('div',{"class":'sidearm-schedule-game-opponent-text'})
    dtags = soup.find_all('div',{"class":'sidearm-schedule-game-opponent-date'})
    rtags = soup.find_all('div',{'class':'sidearm-schedule-game-result'})
    
    # get result tag
    for rtag in rtags[1::2]:
        game_result = rtag.find('span')
        if game_result:
            game_results = rtag.text
            
            #store names
            result_list.append(game_results.strip().replace('\n'," "))
            # print(game_results.strip().replace('\n'," ")+", year: "+schedule_url[-4:])
            
    # get opponent tag
    for otag in otags[1::2]:
        opponent = otag.find('span')
        if opponent:
            opponents = otag.text

            # store names
            opponent_list.append(opponents.strip().replace("\n"," "))

            # print(opponent_names.strip())
        year_list.append(schedule_url[-4:])
    # get date tag
    for dtag in dtags:
        game_date = dtag.find('span')
        if game_date:
            game_dates = dtag.text
            
            # store names
            date_list.append(game_dates.strip().replace("\n"," "))
            # print(game_dates.strip())


In [7]:
len(result_list)

220

In [8]:
isu_hist_schedule['opponent'] = opponent_list
isu_hist_schedule['game_full_date'] = date_list
isu_hist_schedule['game_year'] = year_list
isu_hist_schedule['game_result'] = result_list

isu_hist_schedule[['game_date','game_full_date']] = isu_hist_schedule['game_full_date'].str.split("(",expand=True)
isu_hist_schedule[['game_day_of_week','game_time']] = isu_hist_schedule['game_full_date'].str.split(")",expand=True)
isu_hist_schedule[['game_result','game_score']] = isu_hist_schedule['game_result'].str.split(',',expand=True)

In [9]:
isu_hist_schedule = isu_hist_schedule.replace('OT W',"W")
isu_hist_schedule = isu_hist_schedule.replace('OT L',"L")
isu_hist_schedule = isu_hist_schedule.replace('2OT W',"W")
isu_hist_schedule = isu_hist_schedule.replace('2OT L',"L")
isu_hist_schedule = isu_hist_schedule.replace('4-3 PKs T',"T")
isu_hist_schedule = isu_hist_schedule.replace("W(3-2 in PK's) T","T")
isu_hist_schedule = isu_hist_schedule.replace("","N/A")

In [10]:
isu_hist_schedule['game_result'].value_counts()

L      118
W       70
T       19
N/A     13
Name: game_result, dtype: int64

In [11]:
pd.crosstab(index = isu_hist_schedule['game_year'], columns = isu_hist_schedule['game_result'])

game_result,L,N/A,T,W
game_year,,,,
2009,11,0,3,5
2010,12,0,1,8
2011,13,2,0,4
2012,9,2,3,9
2013,10,2,0,9
2014,9,0,3,5
2015,10,5,1,6
2016,11,2,2,7
2017,11,0,1,7


In [12]:
pd.crosstab(index = isu_hist_schedule['game_day_of_week'], columns = isu_hist_schedule['game_result'])

game_result,L,N/A,T,W
game_day_of_week,,,,
Fri,39,1,7,28
Mon,1,1,0,0
Sat,10,4,3,5
Sun,45,3,9,28
Thu,10,0,0,5
Tue,1,3,0,0
Wed,12,1,0,4


## Indiana State Roster Scrape

In [13]:
url = 'https://gosycamores.com/sports/womens-soccer/roster/2021'

In [14]:
page = urlopen(url)
html = page.read().decode('utf-8')
soup = BeautifulSoup(html, "html.parser")

In [15]:
isu_roster_21 = pd.DataFrame()

In [16]:
last_names = soup.find_all('span',{'class':"sidearm-roster-player-last-name"})
first_names = soup.find_all('span',{'class':"sidearm-roster-player-first-name"})
player_classes = soup.find_all('span',{'class':"sidearm-roster-player-academic-year"})

In [17]:
lname_list = []
for last_name in last_names:
    lname_list.append(last_name.text)
    
fname_list = []
for first_name in first_names:
    fname_list.append(first_name.text)
    
class_list = []
for player_class in player_classes:
    class_list.append(player_class.text)


In [18]:
class_list = class_list[-28:]
lname_list = lname_list[:-3]
fname_list = fname_list[:-3]

In [19]:
len(lname_list)

28

In [20]:
isu_roster_21['last_name'] = lname_list
isu_roster_21['first_name'] = fname_list
isu_roster_21['class'] = class_list

In [21]:
isu_roster_21['class'].value_counts(normalize=True)

Jr.    0.321429
So.    0.285714
Fr.    0.285714
Sr.    0.071429
5th    0.035714
Name: class, dtype: float64

In [22]:
20/35

0.5714285714285714

## ISU 2020 roster

In [23]:
url = 'https://gosycamores.com/sports/womens-soccer/roster/2020'

In [24]:
page = urlopen(url)
html = page.read().decode('utf-8')
soup = BeautifulSoup(html, "html.parser")

In [25]:
isu_roster_20 = pd.DataFrame()

In [26]:
last_names = soup.find_all('span',{'class':"sidearm-roster-player-last-name"})
first_names = soup.find_all('span',{'class':"sidearm-roster-player-first-name"})
player_classes = soup.find_all('span',{'class':"sidearm-roster-player-academic-year"})

In [27]:
lname_list = []
for last_name in last_names:
    lname_list.append(last_name.text)
    
fname_list = []
for first_name in first_names:
    fname_list.append(first_name.text)
    
class_list = []
for player_class in player_classes:
    class_list.append(player_class.text)


In [28]:
class_list = class_list[-23:]
lname_list = lname_list[:-2]
fname_list = fname_list[:-2]

In [29]:
isu_roster_20['last_name'] = lname_list
isu_roster_20['first_name'] = fname_list
isu_roster_20['class'] = class_list

isu_roster_20['class'].value_counts(normalize=True)

So.    0.391304
Fr.    0.347826
Sr.    0.130435
Jr.    0.130435
Name: class, dtype: float64

In [31]:
isu_roster_21['returner'] = isu_roster_20['last_name'].isin(isu_roster_21['last_name'])

In [32]:
isu_roster_21['returner'].value_counts()

True     20
False     3
Name: returner, dtype: int64

In [34]:
isu_roster_21

,last_name,first_name,class,returner
0,Hoffman,Tara,Jr.,True
1,Tesmond,Tara,Fr.,True
2,Alexander,Maddie,So.,True
3,Mackey,Alexa,Fr.,True
4,Yankey,Katie,So.,True
5,Lovell,Olivia,Fr.,True
6,Wahlberg,CeCe,Jr.,True
7,Hudnall,Carly,So.,False
8,Jensen,Carlie,So.,True
9,Helling,Maddie,Fr.,True


## Roster Search

#### Team site stems
Illinois: fightingillini.com, 5 coaches, 33 players\
Nebraska: huskers.com, 8 coaches, 35 players \
Indiana St: gosycamores.com, 3 coaches, 28 players\
Mizzou: mutigers.com, 6 coaches, 29 players

In [171]:
url = 'https://mutigers.com/sports/womens-soccer/roster/2021'

page = urlopen(url)
html = page.read().decode('utf-8')
soup = BeautifulSoup(html, "html.parser")

roster = pd.DataFrame()

last_names = soup.find_all('span',{'class':"sidearm-roster-player-last-name"})
first_names = soup.find_all('span',{'class':"sidearm-roster-player-first-name"})
player_classes = soup.find_all('span',{'class':"sidearm-roster-player-academic-year"})

lname_list = []
for last_name in last_names:
    lname_list.append(last_name.text)
    
fname_list = []
for first_name in first_names:
    fname_list.append(first_name.text)
    
class_list = []
for player_class in player_classes:
    class_list.append(player_class.text)


In [172]:
class_list = class_list[-29:]
lname_list = lname_list[:-6]
fname_list = fname_list[:-6]

In [173]:
roster['last_name'] = lname_list
roster['first_name'] = fname_list
roster['class'] = class_list

roster['class'].value_counts(normalize=True)

Fr.      0.275862
Jr.      0.241379
So.      0.206897
Gr.      0.103448
Sr.      0.103448
R-So.    0.068966
Name: class, dtype: float64

In [174]:
roster['class'].value_counts()

Fr.      8
Jr.      7
So.      6
Gr.      3
Sr.      3
R-So.    2
Name: class, dtype: int64

In [175]:
miz_roster = roster

## Search Illinois player

In [142]:
ill_player = str(input("Search player: "))
illinois_roster.loc[illinois_roster['last_name'] == ill_player, 'class']

Search player: Jones


23    Fr.
Name: class, dtype: object

## Search Nebraska player

In [146]:
neb_player = str(input("Search player: "))
nebraska_roster.loc[nebraska_roster['last_name'] == neb_player, 'class']

Search player: Weber


29    Fr.
Name: class, dtype: object

### Corner Kicks

In [153]:
goal_box_list = [2,1,2,2,2,2,2,1,1,1]
penalty_box_list = [4,4,4,4,4,5,4,5,4,5]
result_list = ['clearance','goal kick','clearance', 'clearance','rebound, save','clearance','throw in','rebound, miss','rebound, clearance','rebound, save']

In [154]:
corner_kicks = pd.DataFrame()

In [155]:
corner_kicks['goal_box'] = goal_box_list
corner_kicks['penalty_box'] = penalty_box_list
corner_kicks['total'] = corner_kicks['goal_box'] + corner_kicks['penalty_box']
corner_kicks['result'] = result_list

In [156]:
corner_kicks

,goal_box,penalty_box,total,result
0,2,4,6,clearance
1,1,4,5,goal kick
2,2,4,6,clearance
3,2,4,6,clearance
4,2,4,6,"rebound, save"
5,2,5,7,clearance
6,2,4,6,throw in
7,1,5,6,"rebound, miss"
8,1,4,5,"rebound, clearance"
9,1,5,6,"rebound, save"


In [157]:
corner_kicks['result'].value_counts()

clearance             4
rebound, save         2
rebound, miss         1
rebound, clearance    1
throw in              1
goal kick             1
Name: result, dtype: int64

## Weighted Roster experience

In [268]:
nebraska_roster['class'].value_counts()

Fr.    11
Sr.    11
So.     9
Jr.     4
Name: class, dtype: int64

In [269]:
nebraska_roster_breakdown = nebraska_roster['class'].value_counts(normalize=True)

In [270]:
nebraska_roster_breakdown = nebraska_roster_breakdown.tolist()

In [271]:
experience_list = [0,3,1,2]

In [272]:
nebraska_roster_breakdown

[0.3142857142857143,
 0.3142857142857143,
 0.2571428571428571,
 0.11428571428571428]

In [273]:
experience_values = []
experience = 0
for num1, num2 in zip(nebraska_roster_breakdown,experience_list):
    experience_weight = num1 * num2
    experience_values.append(experience_weight)
    experience = experience + (experience_weight)
    print(experience, " (+",experience_weight,")")

0.0  (+ 0.0 )
0.9428571428571428  (+ 0.9428571428571428 )
1.2  (+ 0.2571428571428571 )
1.4285714285714286  (+ 0.22857142857142856 )


In [263]:
9/33

0.2727272727272727

In [274]:
neb_exp = experience

### Average years of team experience

In [275]:
miz_exp

1.482758620689655

In [276]:
ill_exp

1.4242424242424243

In [277]:
neb_exp

1.4285714285714286

In [278]:
isu_exp

1.2857142857142856

## Stefanie Golan Avg Years Team Experience

In [340]:
url = 'https://gophersports.com/sports/womens-soccer/roster/2012'

page = urlopen(url)
html = page.read().decode('utf-8')
soup = BeautifulSoup(html, "html.parser")

roster = pd.DataFrame()

last_names = soup.find_all('span',{'class':"sidearm-roster-player-last-name"})
first_names = soup.find_all('span',{'class':"sidearm-roster-player-first-name"})
player_classes = soup.find_all('span',{'class':"sidearm-roster-player-academic-year"})

lname_list = []
for last_name in last_names:
    lname_list.append(last_name.text)
    
fname_list = []
for first_name in first_names:
    fname_list.append(first_name.text)
    
class_list = []
for player_class in player_classes:
    class_list.append(player_class.text)


In [341]:
len(fname_list)

20

In [342]:
class_list = class_list[-20:]
lname_list = lname_list[:]
fname_list = fname_list[:]

In [343]:
roster['last_name'] = lname_list
roster['first_name'] = fname_list
roster['class'] = class_list

umn_roster_2012 = roster

In [348]:
for year in range(2012,2021):
    roster_year = 'umn_roster_'+str(year)
    print(roster_year)

umn_roster_2012
umn_roster_2013
umn_roster_2014
umn_roster_2015
umn_roster_2016
umn_roster_2017
umn_roster_2018
umn_roster_2019
umn_roster_2020


In [349]:
rosters = [umn_roster_2012,
umn_roster_2013,
umn_roster_2014,
umn_roster_2015,
umn_roster_2016,
umn_roster_2017,
umn_roster_2018,
umn_roster_2019,
umn_roster_2020]

In [350]:
for roster in rosters:
    print(roster)

    last_name first_name  class
0     Knutson    Kristen    Fr.
1    Parkhill        Cat  R-Sr.
2       Baier     Nicole    Sr.
3         Uhl     Taylor    So.
4       Price    Brianne    So.
5   Helverson      Haley    Fr.
6      Thyken      Katie    So.
7    Phillips      Allie    Sr.
8     Arvesen      Ellie    Fr.
9      Dykema       Elle    Fr.
10      Price    Marissa    Sr.
11     Brandt      Steph  R-Jr.
12    Bagnall     Olivia    Sr.
13   Strahota     Tamara  R-Sr.
14    Roberts      Becca    So.
15     Reinke      Allie    Fr.
16      Bauer     Lauren    Jr.
17      Misel  MacKenzie    Jr.
18    Wodnick     Taylor    So.
19      Voigt     Jackie    Sr.
     last_name first_name  class
0      Knutson    Kristen    So.
1        Hobbs      Tarah    Fr.
2    Dougherty       Mara    Fr.
3     Kolander     Simone    Fr.
4      Schultz     Olivia    Jr.
5    Helverson      Haley    So.
6       Thyken      Katie    Jr.
7     Giddings      Allie    So.
8      Arvesen      Ellie    So